In [1]:
# --- FASE 0: IMPORTAR LIBRERÍAS ---

# Librerías para manipulación y análisis de datos
import pandas as pd
import numpy as np

# Librerías para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Herramientas de Scikit-Learn para modelado y evaluación
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Configuración para mejorar la visualización de los gráficos
%matplotlib inline
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-deep')


# --- FASE 1: PREPROCESAMIENTO Y ANÁLISIS EXPLORATORIO (EDA) ---

# 1.1 Carga de Datos
# Asegúrate de que el archivo "Titanic.xlsx" esté en la misma carpeta que tu notebook,
# o ajusta la ruta según sea necesario.
df_titanic_raw = pd.read_excel("../MyPython/dataset/Titanic.xlsx")

# Creamos una copia para no modificar el original
df_titanic = df_titanic_raw.copy()


# 1.2 Análisis Gráfico (Opcional, pero recomendado)
# Puedes mantener tu código de gráficos aquí si quieres visualizar los datos crudos.


# 1.3 Limpieza, Selección e Ingeniería de Características
print("--- Iniciando preprocesamiento y creación de características ---")

# Rellenar valores nulos en columnas clave
df_titanic['Age'] = df_titanic['Age'].fillna(df_titanic['Age'].median())
df_titanic['Passenger Fare'] = df_titanic['Passenger Fare'].fillna(df_titanic['Passenger Fare'].median())
df_titanic['Port of Embarkation'] = df_titanic['Port of Embarkation'].fillna(df_titanic['Port of Embarkation'].mode()[0])

# INGENIERÍA DE CARACTERÍSTICAS
# Crear 'FamilySize'
df_titanic['FamilySize'] = df_titanic['No of Siblings or Spouses on Board'] + df_titanic['No of Parents or Children on Board'] + 1

# Crear 'Title' a partir de 'Name'
df_titanic['Title'] = df_titanic['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_titanic['Title'] = df_titanic['Title'].fillna('Unknown')
df_titanic['Title'] = df_titanic['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_titanic['Title'] = df_titanic['Title'].replace(['Mlle', 'Ms'], 'Miss')
df_titanic['Title'] = df_titanic['Title'].replace('Mme', 'Mrs')

# CONVERSIÓN DE DATOS
# Convertir la columna objetivo a formato numérico (0 o 1)
df_titanic['Survived'] = df_titanic['Survived'].map({'No': 0, 'Yes': 1})

# Seleccionar las columnas finales que usaremos para el modelo
features_to_keep = ['Survived', 'Passenger Class', 'Sex', 'Age', 'Passenger Fare', 'Port of Embarkation', 'FamilySize', 'Title']
df_processed = df_titanic[features_to_keep]

# Convertir todas las variables categóricas restantes a numéricas usando "dummies"
# Esto crea columnas nuevas para cada categoría (ej. Sex_male, Title_Miss, etc.)
df_final = pd.get_dummies(df_processed, columns=['Passenger Class', 'Sex', 'Port of Embarkation', 'Title'], drop_first=True)

print("Preprocesamiento finalizado. Columnas del dataset final:")
print(df_final.columns)


# --- FASE 2: DIVISIÓN DE DATOS ---

# Separar características (X) y objetivo (y)
X = df_final.drop('Survived', axis=1)
y = df_final['Survived']

# Dividir en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDatos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")


# --- FASE 3: BÚSQUEDA DEL MEJOR MODELO Y OPTIMIZACIÓN ---

print("\n--- Buscando los mejores parámetros para Gradient Boosting... ---")

# Definir los parámetros que GridSearchCV probará
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5]
}

# Configurar y ejecutar la búsqueda
grid_search = GridSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5, # Validación cruzada con 5 pliegues
                           n_jobs=-1, # Usar todos los núcleos del CPU
                           scoring='accuracy')

# Entrenar (esto puede tardar un poco)
grid_trib = grid_search.fit(X_train, y_train)


# --- FASE 4: RESULTADOS FINALES ---

print("\n--- ¡Búsqueda Finalizada! ---")
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor score durante la validación cruzada: {grid_search.best_score_:.4f}")

# Evaluar el modelo optimizado con el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)

print("\n-------------------------------------------------------------")
print(f"🏆 SCORE FINAL OPTIMIZADO: {final_accuracy:.4f} 🏆")
print("-------------------------------------------------------------")

<>:50: SyntaxWarning: invalid escape sequence '\.'
<>:50: SyntaxWarning: invalid escape sequence '\.'
C:\Users\d4vho\AppData\Local\Temp\ipykernel_20160\1236505500.py:50: SyntaxWarning: invalid escape sequence '\.'
  df_titanic['Title'] = df_titanic['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
c:\Users\d4vho\Desktop\TI\Inteligencia de Negocios\MyPython\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


--- Iniciando preprocesamiento y creación de características ---
Preprocesamiento finalizado. Columnas del dataset final:
Index(['Survived', 'Age', 'Passenger Fare', 'FamilySize',
       'Passenger Class_Second', 'Passenger Class_Third', 'Sex_Male',
       'Port of Embarkation_Queenstown', 'Port of Embarkation_Southampton',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare'],
      dtype='object')

Datos divididos: 1047 para entrenamiento, 262 para prueba.

--- Buscando los mejores parámetros para Gradient Boosting... ---

--- ¡Búsqueda Finalizada! ---
Mejores parámetros encontrados: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}
Mejor score durante la validación cruzada: 0.8223

-------------------------------------------------------------
🏆 SCORE FINAL OPTIMIZADO: 0.7710 🏆
-------------------------------------------------------------
